In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
data = pd.read_csv(os.path.join(dirname, "train.csv"))
# data = pd.read_csv(os.path.join(dirname, "test.csv"))

In [3]:
!cat /kaggle/input/house-prices-advanced-regression-techniques/data_description.txt

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

In [4]:
#crating groups of related attributes for convenience
locality = ['MSZoning', 'Street', 'Alley','Neighborhood', 'Condition1', 'Condition2']
lot = ['LotFrontage', 'LotArea', 'LotShape']
masonry=['MasVnrType', 'MasVnrArea', 'Foundation']
land = ['LandContour', 'LandSlope' ]
condition = ['OverallCond', 'ExterCond', 'BsmtCond', 'GarageCond', 'HeatingQC']
quality = ['OverallQual', 'ExterQual', 'BsmtQual', 'GarageQual', 'KitchenQual','PoolQC', 'HeatingQC', 'FireplaceQu']
house=['MSSubClass', 'BldgType', 'HouseStyle', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF','GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd']
utilities=['Utilities', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'Fireplaces', 'GarageCars', 'WoodDeckSF', 'PoolArea']
basement=['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']
fireplace=['FirePlaces', 'FirePlaceQu']
garage=['GarageType', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'GarageYrBlt']
porch=['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
pool=['PoolArea', 'PoolQC']
history=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
roof=['RoofStyle', 'RoofMatl']
exterior=['Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond']
misc=['Fence', 'Foundation', 'MiscFeature', 'MiscVal']
sell=['MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SellPrice']

In [5]:
print("data shape: ", data.shape)
data.head()

data shape:  (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
sale_binned = pd.cut(data.SalePrice,bins=[0, 130000, 250000, 330000,500000, 1000000], labels=["Low", "Med", "High", "Very High", "Exceptional" ] )
subs = make_subplots(1,2, subplot_titles=["Sale Price BoxPlot", "Sale Price Distribution"])
data["SalePrice_binned"] = sale_binned

#make different datasets for earch bin
low_price = data[data["SalePrice_binned"] == "Low"]
med_price = data[data["SalePrice_binned"] == "Med"]
high_price = data[data["SalePrice_binned"] == "High"]
very_high_price = data[data["SalePrice_binned"] == "Very High"]
exceptional_price = data[data["SalePrice_binned"] == "Exceptional"]

subs.add_trace(go.Box(x=data.SalePrice), row=1, col=1)
subs.add_trace(go.Histogram(x=data.SalePrice), row=1,col=2)
subs.show()
px.histogram(sale_binned, title="Sale Price Binned").show()

# Data Cleaning

In [7]:
def show_missing_values(data):
    #Missing values
    missing = pd.DataFrame(data.isnull().sum())
    return missing[missing[0] > 0]
show_missing_values(data)

,0
LotFrontage,259
Alley,1369
MasVnrType,8
MasVnrArea,8
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1


## 1. Locality

In [8]:
locality_data = data[locality].copy()
locality_data.head()

,MSZoning,Street,Alley,Neighborhood,Condition1,Condition2
0,RL,Pave,NaN,CollgCr,Norm,Norm
1,RL,Pave,NaN,Veenker,Feedr,Norm
2,RL,Pave,NaN,CollgCr,Norm,Norm
3,RL,Pave,NaN,Crawfor,Norm,Norm
4,RL,Pave,NaN,NoRidge,Norm,Norm


In [9]:
locality_data.isnull().sum()

MSZoning           0
Street             0
Alley           1369
Neighborhood       0
Condition1         0
Condition2         0
dtype: int64

In [10]:
def clean_locality(locality_data):
    # Allley has huge amount of missing values, alley is supposed to have following classes: 'Grvl', 'Pave' and 'None'
    #we are missing 'None' class. It is a mistake when entering data. Instead of 'None' it is filled with NaN. We can observe that after looking at frequency counts for Alley
    locality_data = locality_data.fillna("None")
    return locality_data

In [11]:
data[locality] = clean_locality(locality_data)

In [12]:
subs = make_subplots(2,3, subplot_titles=locality)
col_no=0
for r in range(1, 3):
    for c in range(1, 4):
        attr = locality[col_no]
        subs.add_trace(go.Histogram(x=locality_data[attr]), row=r, col=c)
        col_no=col_no+1
subs.show()

## 2. Lot

In [13]:
lot_data = data[lot].copy()
lot_data.head()

,LotFrontage,LotArea,LotShape
0,65.0,8450,Reg
1,80.0,9600,Reg
2,68.0,11250,IR1
3,60.0,9550,IR1
4,84.0,14260,IR1


In [14]:
lot_data.isnull().sum()

LotFrontage    259
LotArea          0
LotShape         0
dtype: int64

In [15]:
subplots = make_subplots(1, 3, subplot_titles=("LotShape Count", "LotShape vs LotFrontage", "LotShape vs LotArea"))
subplots.add_trace(go.Histogram(x=lot_data.LotShape), row=1, col=1)
subplots.add_trace(go.Histogram(x=lot_data.LotShape, y=lot_data.LotFrontage, histfunc="avg"), row=1, col=2)
subplots.add_trace(go.Histogram(x=lot_data.LotShape, y=lot_data.LotArea, histfunc="avg"), row=1, col=3)

In [16]:
subs = make_subplots(2, 2, subplot_titles=("Regular", "IR1", "IR2", "IR3"))

idx=0
lotShapes = lot_data.LotShape.unique()
for r in range(1, 3):
    for c in range(1, 3):
        filtered_data = lot_data[lot_data.LotShape == lotShapes[idx]]
        subs.add_trace(go.Scatter(x=filtered_data.LotFrontage, y=filtered_data.LotArea, mode="markers"), row=r, col=c)
        idx=idx+1
subs.show()

In [17]:
def clean_lot(lot_data):
    #now, we compute mean and impute missing values in LotFrontage column
    mean_value = lot_data.LotFrontage.mean()
    lot_data.LotFrontage=lot_data.LotFrontage.fillna(mean_value)
    
    return lot_data

In [18]:
#from above visualizations, we can figure out that we may have 2 outliers when it comes to lotfrontage, that is values with LotFrontage==313
#let's remove them
lot_data = lot_data[lot_data.LotFrontage < 313]
#also remove one possible outlier for LotArea attribuet, that is, LotArea==215
lot_data = lot_data[lot_data.LotArea < 215000]

data = data[(data.LotFrontage < 313) & (data.LotArea <215000)]
data[lot] = clean_lot(lot_data)
#visualizing results
subplots = make_subplots(1, 2, subplot_titles=("LotShape VS LotFrontage", "LotShape VS LotArea"))
subplots.add_trace(go.Histogram(x=lot_data.LotShape, y = lot_data.LotFrontage, histfunc='avg'), row=1, col=1)
subplots.add_trace(go.Histogram(x=lot_data.LotShape, y=lot_data.LotArea, histfunc="avg"), row=1, col=2)

## 3. Masonry

In [19]:
masonry_data = data[masonry].copy()
print("Missing values: ", masonry_data.isnull().sum())

print("Unique values for Masonry Vaneer Type: ", masonry_data.MasVnrType.unique())

Missing values:  MasVnrType    6
MasVnrArea    6
Foundation    0
dtype: int64
Unique values for Masonry Vaneer Type:  ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]


In [20]:
masonry_data.columns

Index(['MasVnrType', 'MasVnrArea', 'Foundation'], dtype='object')

In [21]:
def clean_masonry(masonry_data):
    #fill missing masonry vaneer area with mean values
    masonry_data.MasVnrArea = masonry_data.MasVnrArea.fillna(masonry_data.MasVnrArea.mean())
    mode =  masonry_data.MasVnrType.mode()
    print("Mode: ", mode[0])
    masonry_data.MasVnrType= masonry_data.MasVnrType.fillna(mode[0])
    return masonry_data

In [22]:
data[masonry] = clean_masonry(masonry_data)

Mode:  None


In [23]:
subs = make_subplots(3,3, subplot_titles=("Vaneer Type", "Vaneer Type Vs avg SalePrice", "Foundation", "Vaneer Area vs SalePrice", "Foundation vs SalePrice", "Foundation vs Vaneer Area", "Vaneer Type Vs Vaneer Area"))
subs.add_trace(go.Histogram(x=masonry_data.MasVnrType), row=1, col=1)
subs.add_trace(go.Histogram(x=masonry_data.MasVnrType, y=data.SalePrice, histfunc="avg"), row=1, col=2)
subs.add_trace(go.Histogram(x=masonry_data.Foundation, y=masonry_data.MasVnrArea), row=1, col=3)
subs.add_trace(go.Scatter(x=masonry_data.MasVnrArea, y=data.SalePrice, mode="markers"), row=2, col=1)
subs.add_trace(go.Histogram(x=masonry_data.Foundation, y=data.SalePrice, histfunc="avg"), row=2, col=2)
subs.add_trace(go.Histogram(x=masonry_data.Foundation, y=masonry_data.MasVnrArea, histfunc="avg"), row=2, col=3)
subs.add_trace(go.Histogram(x=masonry_data.MasVnrType, y=masonry_data.MasVnrArea, histfunc="avg"), row=3, col=1)
subs.show()

## 4. Basement

In [24]:
basement_data = data[basement].copy()
basement_data.head()

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
0,Gd,TA,No,GLQ,706,Unf,0,150,856
1,Gd,TA,Gd,ALQ,978,Unf,0,284,1262
2,Gd,TA,Mn,GLQ,486,Unf,0,434,920
3,TA,Gd,No,ALQ,216,Unf,0,540,756
4,Gd,TA,Av,GLQ,655,Unf,0,490,1145


In [25]:
basement_data.isnull().sum()

BsmtQual        31
BsmtCond        31
BsmtExposure    32
BsmtFinType1    31
BsmtFinSF1       0
BsmtFinType2    32
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
dtype: int64

they all seem to have almost same amount of missing samples. Is there any relation between them?

In [26]:
def clean_basement(basement_data):
    bsmt_cat = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1']
    bsmt_num = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']

    basement_data[bsmt_cat] = basement_data[bsmt_cat].fillna("None")
    basement_data[bsmt_num] = basement_data[bsmt_num].fillna(0)
    return basement_data

In [27]:
data[basement] = clean_basement(basement_data)

In [28]:
basement_data.columns

Index(['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'],
      dtype='object')

In [29]:
subs = make_subplots(3, 3, subplot_titles=basement)
col_no=0
for r in range(1, 4):
    for c in range(1, 4):
        attr = basement[col_no]
        subs.add_trace(go.Histogram(x=basement_data[attr], y=data.SalePrice, histfunc="avg"), row=r, col=c)
        col_no=col_no+1
subs.show()

## 5. Utilites

In [30]:
utilities_data = data[utilities].copy()
utilities_data.isnull().sum()

Utilities     0
Heating       0
CentralAir    0
Electrical    1
Functional    0
Fireplaces    0
GarageCars    0
WoodDeckSF    0
PoolArea      0
dtype: int64

In [31]:
def clean_utilities(utilities_data):
    utilities_data.Utilities = utilities_data.Utilities.fillna(utilities_data.Utilities.mode()[0])
    utilities_data.Electrical = utilities_data.Electrical.fillna(utilities_data.Electrical.mode()[0])
    return utilities_data

In [32]:
data[utilities] = clean_utilities(utilities_data)

In [33]:
subs = make_subplots(3,3, subplot_titles=utilities)
col_no=0
for r in range(1, 4):
    for c in range(1, 4):
        attr = utilities[col_no]
        subs.add_trace(go.Histogram(x=utilities_data[attr]), row=r, col=c)
        col_no=col_no+1
subs.show()

## 6. Quality

In [34]:
qual_data = data[quality].copy()
qual_data.isnull().sum()

OverallQual       0
ExterQual         0
BsmtQual          0
GarageQual       74
KitchenQual       0
PoolQC         1193
HeatingQC         0
FireplaceQu     600
dtype: int64

In [35]:
qual_data.PoolQC.unique() # other two also have same results, they have missing values where there is no Pool, Garage or Fireplace

array([nan, 'Ex', 'Fa', 'Gd'], dtype=object)

In [36]:
def clean_quality(qual_data):
    qual_data = qual_data.fillna("None")
    return qual_data

In [37]:
data[quality] = clean_quality(qual_data)

In [38]:
subs = make_subplots(3,3, subplot_titles=quality)
col_no=0
for r in range(1, 4):
    for c in range(1, 4):
        if(col_no < len(quality)):
            attr = quality[col_no]
            subs.add_trace(go.Histogram(x=qual_data[attr]), row=r, col=c)
            col_no=col_no+1
subs.show()

## 7. Garage

In [39]:
garage_data = data[garage].copy()
garage_data.isnull().sum()

GarageType      74
GarageFinish    74
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond      74
PavedDrive       0
GarageYrBlt     74
dtype: int64

In [40]:
def clean_garage(garage_data):
    garage_cats = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'GarageYrBlt']
    garage_nums = ['GarageCars', 'GarageArea']
    
    garage_data["GarageYrBlt"] = garage_data['GarageYrBlt'].fillna(garage_data.GarageYrBlt.mode()[0]) 
    garage_data[garage_cats] = garage_data[garage_cats].fillna("None")
    garage_data[garage_nums] = garage_data[garage_nums].fillna(0)
    return garage_data

In [41]:
data[garage] = clean_garage(garage_data)

## 8. Misc Features

In [42]:
misc_data = data[misc]
misc_data.isnull().sum()

Fence           970
Foundation        0
MiscFeature    1159
MiscVal           0
dtype: int64

In [43]:
def clean_misc(misc_data):
    misc_data = misc_data.fillna("None")
    return misc_data

In [44]:
data[misc] = clean_misc(misc_data)

In [45]:
subs = make_subplots(2,2, subplot_titles=misc)
col_no=0
for r in range(1, 3):
    for c in range(1, 3):
        if(col_no < len(misc)):
            attr = misc[col_no]
            subs.add_trace(go.Histogram(x=misc_data[attr]), row=r, col=c)
            col_no=col_no+1
subs.show()

## 9. Exterior

In [46]:
ext_data = data[exterior]
ext_data.isnull().sum()

Exterior1st    0
Exterior2nd    0
ExterQual      0
ExterCond      0
dtype: int64

In [47]:
ext_data.head()

,Exterior1st,Exterior2nd,ExterQual,ExterCond
0,VinylSd,VinylSd,Gd,TA
1,MetalSd,MetalSd,TA,TA
2,VinylSd,VinylSd,Gd,TA
3,Wd Sdng,Wd Shng,TA,TA
4,VinylSd,VinylSd,Gd,TA


In [48]:
def clean_exterior(ext_data):
    ext_mode = ext_data[exterior].mode()
    ext_data = ext_data.fillna({'Exterior1st': ext_mode.Exterior1st[0], 'Exterior2nd': ext_mode.Exterior2nd[0], 'ExterQual': ext_mode.ExterQual[0], 'ExterCond': ext_mode.ExterCond[0] })
    return ext_data

In [49]:
data[exterior] = clean_exterior(ext_data)

# Analysis
In this section, we ask two questions:
1. Why:- why these houses have low prices and other one's have high prices, by comparing them based on best predictors for the SalePrice
2. Which:- Which Features affect the data
3. How:- How do these features affect the data, they cause pricing to increase or decrease, or change in other variables.

## 1. Low Price houses
let's take a look at what low price houses look like. It will also help us in validating if the data is correct for such data class

In [50]:
housedata_low = low_price[house].copy()
housedata_low["SalePrice"]= data.SalePrice
housedata_low.head()

,MSSubClass,BldgType,HouseStyle,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,SalePrice
8,50,1Fam,1.5Fin,1022,752,0,1774,0,0,2,0,2,2,8,129900.0
9,190,2fmCon,1.5Unf,1077,0,0,1077,1,0,1,0,2,2,5,118000.0
10,20,1Fam,1Story,1040,0,0,1040,1,0,1,0,3,1,5,129500.0
17,90,Duplex,1Story,1296,0,0,1296,0,0,2,0,2,2,6,90000.0
23,120,TwnhsE,1Story,1060,0,0,1060,1,0,1,0,3,1,6,129900.0


In [51]:
import plotly.figure_factory as ff

df_corr = housedata_low.corr()

x = list(df_corr.columns)
y = list(df_corr.index)
z = np.array(df_corr)

fig = ff.create_annotated_heatmap(
    z,
    x = x,
    y = y ,
    annotation_text = np.around(z, decimals=2),
    hoverinfo='z',
    colorscale='Viridis'
    )
fig.show()

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



we can see the key predictors are all related to the area, which means generally greater the area greater will be the price of the house. Other than that, there are few autocorrelated variables on the variables explaining area.

**Key Predictors:** TotalRoomsAboveGround, BeedroomsAboveGround, GroundLivingArea, 1stFloorSquareFeetArea

**Interrelated Variables:** 
1. TotalRoomsAboveGround: 1stFloorSF, 2ndFloorSF, GrLivArea, FullBath, Bedroom, Kitchen
2. Beedroom: 1st and 2nd FloorSF,  GrLivArea, FullBath
3. GroundLivingArea: MSSubClass, 1st and 2nd FloorSF
4. 1stFloorSF: none

**We will only look out for these variables:** 1stFloorSF, 2ndFloorSF, GrLivArea, FullBath, Bedroom, Kitchen

In [52]:
features = ["TotRmsAbvGrd", "BedroomAbvGr", "1stFlrSF", "GrLivArea"]
subs = make_subplots(1, 4, subplot_titles=[x + " VS SalePrice" for x in features])
col_no=0

for r in range(1, 2):
    for c in range(1, 5):
        attr = features[col_no]
        plt = go.Histogram(y=housedata_low.SalePrice, x=housedata_low[attr]) if col_no < 2 else go.Scatter(x=housedata_low.SalePrice, y=housedata_low[attr], mode="markers")
        subs.add_trace(plt,row=r, col=c)
        col_no +=1
subs.show()

it seems, Ground Living Area and 1st Floor Square Feet area has positive correlation with Sale Price

Looking closely at data

In [53]:
features = ["1stFlrSF", "2ndFlrSF", "GrLivArea"]
subs = make_subplots(1, 3, subplot_titles=[x + " VS Total Rooms" for x in features])
col_no=0

for r in range(1, 2):
    for c in range(1, 4):
        attr = features[col_no]
        subs.add_trace(go.Histogram(x=housedata_low.TotRmsAbvGrd, y=housedata_low[attr], histfunc="avg"),row=r, col=c)
        col_no +=1
subs.show()

After looking at data closely, we can see that, features that define house properties are highly interrelated, such as Total Rooms and features describing area. Also Total rooms and Bedrooms, Bath and Kitchen. 
Thankfully decision trees can do automatic feature selection. That's why we will leave the rest as it is

### The same can be done between data of other SalePrice classes and other feature groups to derive maximum knowledge about the data
for now, we will not be focusing on instead we will focus on fitting the model using Random Forest and Boosting techniques such as AdaBoost and XGboost

## Fitting the Model

In [54]:
data = data.drop("SalePrice_binned", axis=1)

In [55]:
categorical_features = []
numerical_features = []
for x in data.dtypes.iteritems():
    if(x[1] == "object" or x[1] == "category"):  categorical_features.append(x[0])
    else: numerical_features.append(x[0])
print(categorical_features)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


before we begin encoding categorical variables, let's make sure there are'nt any variables that have same category names

In [56]:
alist = []
for x in categorical_features:
    for y in data[x].unique():
        alist.append(y)
print("Total new columns made when encoded: ", len(alist))

Total new columns made when encoded:  257


In [57]:
from collections import Counter

counts = dict(Counter(alist))
duplicates = {key:value for key, value in counts.items() if value > 1}
print(duplicates)

{'Pave': 2, 'Grvl': 2, 'None': 15, 'Mod': 2, 'Norm': 2, 'Feedr': 2, 'Artery': 2, 'RRNn': 2, 'PosN': 2, 'PosA': 2, 'VinylSd': 2, 'MetalSd': 2, 'Wd Sdng': 2, 'BrkFace': 3, 'HdBoard': 2, 'AsbShng': 2, 'Stucco': 2, 'Plywood': 2, 'AsphShn': 2, 'Stone': 4, 'ImStucc': 2, 'CBlock': 3, 'Gd': 11, 'TA': 9, 'Ex': 9, 'Fa': 10, 'Po': 6, 'GLQ': 2, 'ALQ': 2, 'Unf': 3, 'Rec': 2, 'LwQ': 2, 'BLQ': 2, 'Y': 2, 'N': 2}


and, we have variables with same category/class names. actually lots of them. 

what do we do? let's append each category with it's respective variable name, when encoding. 

such that, it becomes, `variable1_Fa` and `variable2_Fa` and we can differentiate which one is of which variable

In [58]:
#to cross check number of columns after encoding
total_features = 81 #including numerical and categorical features
total_categories = 257 #summing up all categories in all categorical features
total_categorical_features = 43 #all categorical features MSZoning, Condition1 etc.
print("Correct number of features after transformation: ", total_features + total_categories - total_categorical_features)

Correct number of features after transformation:  295


In [59]:
from sklearn.preprocessing import OneHotEncoder

# enc = OneHotEncoder()
# enc.fit(data[categorical_features])

def transform_features(data):
    enc = OneHotEncoder()

    fitted_values = enc.fit_transform(data[categorical_features]).toarray()
    cats = []
    
    for idx, category in enumerate( enc.categories_):
        for c in category:
            try:
                cats.append(categorical_features[idx]+"_"+c)
            except:
                print("Error while transforming: ", categorical_features[idx], " " , c)
    data[cats] = fitted_values
    return data

In [60]:
transformed_data = transform_features(data.copy())
transformed_data = transformed_data.drop(categorical_features, axis=1)
transformed_data[['Functional_Sev', 'RoofStyle_Shed', 'MiscFeature_Gar2', 'MSZoning_None', 'KitchenQual_None']] = 0

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [61]:
print(transformed_data.shape)
transformed_data.head()

(1198, 300)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_Fin,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,Ga

In [62]:
from sklearn.model_selection import train_test_split

y = transformed_data.SalePrice
x = transformed_data.drop("SalePrice", axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

## Random Forest

In [63]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=5,oob_score=True, bootstrap=True)
regr.fit(X_train, y_train)
output = cross_validate(regr, X_train, y_train, cv=5, return_estimator=True, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.8s finished


In [64]:
regr.score(X_test, y_test)

0.7876743348165686

In [65]:
importances = []
for idx,estimator in enumerate(output['estimator']):
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X_train.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
#     print(estimator.get_params())
    importances.append(feature_importances)

In [66]:
print("Avaregae Score: ", sum(output['test_score'])/len(output['test_score']))    

Avaregae Score:  0.8557457681514931


## AdaBoost

In [67]:
from sklearn.ensemble import AdaBoostRegressor
ada_regr = AdaBoostRegressor(base_estimator=regr, random_state=0, n_estimators=10)
ada_regr.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=RandomForestRegressor(max_depth=5,
                                                       oob_score=True),
                  n_estimators=10, random_state=0)

In [68]:
ada_regr.score(X_test, y_test)

0.7852061849954733

In [69]:
#cross-validation
output = cross_validate(ada_regr, X_train, y_train, cv=5, return_estimator=True, verbose=1)
print("Avaregae Score: ", sum(output['test_score'])/len(output['test_score']))    

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Avaregae Score:  0.8868629560874831


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   36.1s finished


## XGBoost

In [70]:
from sklearn import ensemble
xg_regr = reg = ensemble.GradientBoostingRegressor(n_estimators=500, max_depth=5, learning_rate=0.01, loss="squared_error")
xg_regr.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=5, n_estimators=500)

In [71]:
xg_regr.score(X_test, y_test)

0.8223135234341646

In [72]:
#cross-validation
output = cross_validate(xg_regr, X_train, y_train, cv=5, return_estimator=True, verbose=1)
print("Avaregae Score: ", sum(output['test_score'])/len(output['test_score']))    

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Avaregae Score:  0.8904868625391114


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   21.7s finished


## Submission

In [73]:
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [74]:
def clean_data(data):
    data[locality] = clean_locality(data[locality])
    data[lot] = clean_lot(data[lot])
    data[masonry] = clean_masonry(data[masonry])
    data[basement] = clean_basement(data[basement])
    data[utilities] = clean_utilities(data[utilities])
    data[quality] = clean_quality(data[quality])
    data[garage] = clean_garage(data[garage])
    data[misc] = clean_misc(data[misc])
    data[exterior] = clean_exterior(data[exterior])
    return data

In [75]:
test_data = clean_data(test_data)
test_data.shape

Mode:  None


(1459, 80)

In [76]:
show_missing_values(test_data)

,0
BsmtFullBath,2
BsmtHalfBath,2
Functional,2
SaleType,1


In [77]:
modes = test_data[['BsmtFullBath', 'BsmtHalfBath', 'Functional', 'SaleType']].mode()

test_data = test_data.fillna({'BsmtFullBath': modes.BsmtFullBath[0], 'BsmtHalfBath': modes.BsmtHalfBath[0] , 'Functional': modes.Functional[0] , 'SaleType': modes.SaleType[0] })
show_missing_values(test_data)

,0


In [78]:
transformed_test_data = transform_features(test_data.copy())
transformed_test_data.drop(categorical_features, axis=1, inplace=True)
print(transformed_test_data.shape)
transformed_test_data.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



(1459, 287)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_None,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_None,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_Fin,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Non

In [79]:
a_set = set(transformed_data)
b_set = set(transformed_test_data)

missing_columns = list( a_set ^ b_set)
missing_columns

['RoofMatl_Roll',
 'GarageQual_Ex',
 'RoofMatl_Membran',
 'Exterior1st_Stone',
 'Condition2_RRNn',
 'SalePrice',
 'Exterior1st_ImStucc',
 'Heating_OthW',
 'Electrical_Mix',
 'HouseStyle_2.5Fin',
 'PoolQC_Fa',
 'Exterior2nd_Other',
 'MiscFeature_TenC']

In [80]:
not_in_test_data = []
not_in_data = []

for x in missing_columns:
    if x not in transformed_test_data.columns: not_in_test_data.append(x)
    elif x not in transformed_data.columns: not_in_data.append(x)

print(not_in_data)

if 'SalePrice' in not_in_test_data: not_in_test_data.remove('SalePrice')
transformed_test_data[not_in_test_data] = 0

[]


In [81]:
results = xg_regr.predict(transformed_test_data)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.




In [82]:
px.histogram(results)

In [83]:
sub_frame = pd.DataFrame({"Id": transformed_test_data.Id, "SalePrice": results})
sub_frame.to_csv('submission.csv', index=False)